<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectifs" data-toc-modified-id="Objectifs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectifs</a></span></li><li><span><a href="#Dev" data-toc-modified-id="Dev-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dev</a></span></li><li><span><a href="#Industrialisation" data-toc-modified-id="Industrialisation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Industrialisation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Script" data-toc-modified-id="Script-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Script</a></span></li></ul></li></ul></li><li><span><a href="#Analyse-de-la-feature" data-toc-modified-id="Analyse-de-la-feature-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse de la feature</a></span><ul class="toc-item"><li><span><a href="#Stations-avec-le-plus-grand-nombre-de-non-prise-de-vélo-consécutif" data-toc-modified-id="Stations-avec-le-plus-grand-nombre-de-non-prise-de-vélo-consécutif-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Stations avec le plus grand nombre de non prise de vélo consécutif</a></span></li></ul></li></ul></div>

In [19]:
from datetime import datetime

import pandas as pd
import polars as pl
import seaborn as sns

from vcub_keeper.config import ROOT_DATA_CLEAN
from vcub_keeper.reader.reader import read_time_serie_activity
from vcub_keeper.transform.features_factory import get_consecutive_no_transactions_out, get_transactions_out
from vcub_keeper.visualisation import plot_station_activity

sns.set_style("whitegrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 700
pl.Config.set_tbl_rows(700)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


polars.config.Config

## Objectifs

- Permettre de savoir depuis combien de temps un vélo n'a pas été pris par station. Plus ce nombre est grand plus il y a de chance que la station soit HS. Cf https://github.com/armgilles/vcub_keeper/issues/10
- Industrialisation de la function.
- Analyse de la feature

## Dev

In [20]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = ts_activity.with_columns(get_transactions_out())
ts_activity = ts_activity.collect()

In [4]:
len(ts_activity)

12794673

In [5]:
def old_fonction_pd(data: pd.DataFrame) -> pd.DataFrame:
    """ """

    data["have_data"] = 1
    data.loc[data["available_stands"].isna(), "have_data"] = 0

    data["consecutive_no_transactions_out"] = data.groupby(
        [
            "station_id",
            (data["available_bikes"] < 3).cumsum(),  # 3 for 2 available_bikes
            (data["have_data"] == 0).cumsum(),
            (data["status"] == 0).cumsum(),
            (data["transactions_out"] > 0).cumsum(),
        ]
    ).cumcount()

    data["consecutive_no_transactions_out"] = data["consecutive_no_transactions_out"].fillna(0)

    data.loc[data["available_stands"].isna(), "consecutive_no_transactions_out"] = 0

    data = data.drop("have_data", axis=1)
    return data

In [6]:
def new_fonction_pl(data: pl.DataFrame) -> pl.DataFrame:
    """ """
    data = (
        data.with_columns(
            pl.when(
                (pl.col("transactions_out") >= 1)
                | (pl.col("status") == 0)
                | (pl.col("available_stands") <= 2)
                | (pl.col("available_stands").is_null())
            )
            .then(0)
            .otherwise(1)
            .alias("logic")
        )
        .with_columns(
            pl.int_ranges(pl.struct("station_id", "logic").rle().struct.field("len"))
            .flatten()
            .alias("consecutive_no_transactions_out")
            + 1
        )
        .with_columns(
            pl.when(pl.col("logic") == 1)
            .then(pl.col("consecutive_no_transactions_out"))
            .otherwise(0)
            .alias("consecutive_no_transactions_out")
        )
        .drop("logic")
    )

    return data

In [302]:
ts_activity = get_transactions_out(ts_activity)

In [303]:
ts_activity.head()

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
u64,datetime[ns],f64,f64,f64,f64,f64,f64
1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0
1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0
1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0
1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0
1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0


In [7]:
ts_activity_pd = ts_activity.to_pandas()

In [8]:
%%timeit -r 10
old_fonction_pd(ts_activity_pd)
# 2.06 s ± 29.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)

2.06 s ± 29.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [14]:
%%timeit -r 10
new_fonction_pl(ts_activity)
# 257 ms ± 806 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)

# ts_activity = new_fonction_pl(ts_activity)

264 ms ± 6.09 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [15]:
%%timeit -r 10
ts_activity.with_columns(get_consecutive_no_transactions_out())

273 ms ± 1.53 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [22]:
ts_activity = ts_activity.with_columns(get_consecutive_no_transactions_out())

Analyse des données lorsqu'il manque des données.

In [23]:
station_id = 145
start_date = "2018-12-29"
end_date = "2019-01-03"

# ts_activity[
#     (ts_activity["station_id"] == station_id) & (ts_activity["date"] >= start_date) & (ts_activity["date"] <= end_date)
# ]
ts_activity.filter((pl.col("station_id") >= station_id) & (pl.col("date") <= datetime(2018, 12, 31, 13, 0, 0))).tail(15)

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u8,datetime[ns],i8,i8,u8,i64,i8,i64,i64
244,2018-12-28 22:00:00,18,12,1,1,1,3,0
244,2018-12-28 22:10:00,18,12,1,0,0,0,1
244,2018-12-28 22:20:00,18,12,1,0,0,0,2
244,2018-12-28 22:30:00,18,12,1,0,0,0,3
244,2018-12-28 22:40:00,19,11,1,0,1,1,0
244,2018-12-28 22:50:00,20,10,1,0,1,1,0
244,2018-12-28 23:00:00,20,10,1,0,0,0,1
244,2018-12-28 23:10:00,19,11,1,1,0,1,2
244,2018-12-28 23:20:00,18,12,1,1,0,1,3


In [24]:
ts_activity.filter(pl.col("available_stands").is_null()).select("consecutive_no_transactions_out").unique()

consecutive_no_transactions_out
i64


Analyse lorsque qu'un station est déconnecté.

In [25]:
station_id = 172
ts_activity.filter((pl.col("station_id") == station_id) & (pl.col("date") >= datetime(2020, 5, 11, 6, 30))).head(10)

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u8,datetime[ns],i8,i8,u8,i64,i8,i64,i64
172,2020-05-11 06:30:00,28,0,0,0,0,0,0
172,2020-05-11 06:40:00,28,0,1,0,0,0,1
172,2020-05-11 06:50:00,28,0,1,0,0,0,2
172,2020-05-11 07:00:00,28,0,1,0,0,0,3
172,2020-05-11 07:10:00,24,4,1,4,0,4,4
172,2020-05-11 07:20:00,24,4,1,0,0,0,5
172,2020-05-11 07:30:00,24,4,1,0,0,0,6
172,2020-05-11 07:40:00,24,4,1,0,0,0,7
172,2020-05-11 07:50:00,24,4,1,0,0,0,8


## Industrialisation

In [8]:
from vcub_keeper.transform.features_factory import get_consecutive_no_transactions_out

In [26]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = (
    ts_activity.with_columns(get_transactions_out()).with_columns(get_consecutive_no_transactions_out())
).collect()

In [27]:
# On regarde si la fonction prend en compte le nombre de Vcub dispo pour
# avancer dans consecutive_no_transactions_out
# new = ts_activity[(ts_activity['station_id'] == 22) &
#                  (ts_activity['date'] >= "2019-08-14 08:00:00")].head(60)

new = ts_activity.filter((pl.col("station_id") == 22) & (pl.col("date") >= datetime(2019, 11, 28))).head(60)

new

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u8,datetime[ns],i8,i8,u8,i64,i8,i64,i64
22,2019-11-28 00:00:00,31,2,1,0,0,0,2
22,2019-11-28 00:10:00,31,2,1,0,0,0,3
22,2019-11-28 00:20:00,31,2,1,0,0,0,4
22,2019-11-28 00:30:00,31,2,1,0,0,0,5
22,2019-11-28 00:40:00,31,2,1,0,0,0,6
22,2019-11-28 00:50:00,31,2,1,0,0,0,7
22,2019-11-28 01:00:00,31,2,1,0,0,0,8
22,2019-11-28 01:10:00,31,2,1,0,0,0,9
22,2019-11-28 01:20:00,31,2,1,0,0,0,10


#### Script 

In [28]:
from vcub_keeper.reader.reader import read_time_serie_activity
from vcub_keeper.transform.features_factory import get_consecutive_no_transactions_out, get_transactions_out

In [29]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = (
    ts_activity.with_columns(get_transactions_out()).with_columns(get_consecutive_no_transactions_out())
).collect()

In [30]:
ts_activity.tail(10)

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u8,datetime[ns],i8,i8,u8,i64,i8,i64,i64
251,2020-08-28 10:20:00,28,12,1,0,1,1,0
251,2020-08-28 10:30:00,28,12,1,0,0,0,1
251,2020-08-28 10:40:00,28,12,1,0,0,0,2
251,2020-08-28 10:50:00,28,12,1,0,0,0,3
251,2020-08-28 11:00:00,28,12,1,0,0,0,4
251,2020-08-28 11:10:00,28,12,1,0,0,0,5
251,2020-08-28 11:20:00,28,12,1,0,0,0,6
251,2020-08-28 11:30:00,26,14,1,2,0,2,7
251,2020-08-28 11:40:00,26,14,1,0,0,0,8


## Analyse de la feature

### Stations avec le plus grand nombre de non prise de vélo consécutif

In [31]:
grp_station = ts_activity.group_by("station_id").agg(pl.col("consecutive_no_transactions_out").max())
grp_station = grp_station.sort("consecutive_no_transactions_out", descending=True)

In [32]:
grp_station.head(10)

station_id,consecutive_no_transactions_out
u8,i64
160,6475
180,3443
161,3431
182,3005
150,2698
168,2215
87,2200
81,2167
92,2103


**Certaines stations sont très inactives (travaux ?)**

In [ ]:
station_id = 172
start_date = "2020-05-03"
end_date = "2020-05-21"
tt = plot_station_activity(
    ts_activity.to_pandas(),
    station_id=station_id,
    features_to_plot=["available_bikes", "available_stands", "consecutive_no_transactions_out", "status"],
    start_date=start_date,
    end_date=end_date,
    return_data=True,
)

# Test

In [11]:
from vcub_keeper.reader.reader import read_time_serie_activity
from vcub_keeper.transform.features_factory import get_consecutive_no_transactions_out, get_transactions_out

In [336]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

ts_activity = get_transactions_out(pl.from_pandas(ts_activity))
ts_activity = get_consecutive_no_transactions_out(ts_activity)

/home/gillesa/github/vcub_keeper/src/vcub_keeper/reader/reader.py:126: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [337]:
ts_activity.schema

Schema([('station_id', UInt64),
        ('date', Datetime(time_unit='ns', time_zone=None)),
        ('available_stands', Float64),
        ('available_bikes', Float64),
        ('status', Float64),
        ('transactions_in', Float64),
        ('transactions_out', Float64),
        ('transactions_all', Float64),
        ('consecutive_no_transactions_out', Int64)])

In [338]:
ts_activity.head()

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u64,datetime[ns],f64,f64,f64,f64,f64,f64,i64
1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0,1
1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0,2
1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0,3
1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0,4
1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0,5


In [339]:
ts_activity.head(8)

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out
u64,datetime[ns],f64,f64,f64,f64,f64,f64,i64
1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0,1
1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0,2
1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0,3
1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0,4
1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0,5
1,2018-12-01 01:00:00,28.0,4.0,1.0,0.0,1.0,1.0,0
1,2018-12-01 01:10:00,28.0,4.0,1.0,0.0,0.0,0.0,1
1,2018-12-01 01:20:00,28.0,4.0,1.0,0.0,0.0,0.0,2


In [32]:
# ts_activity[(ts_activity["transactions_out"] > 1) & (ts_activity["date"] == "2017-07-09 11:09:04")].head()
ts_activity[(ts_activity["date"] >= "2017-07-09 11:04:04") & (ts_activity["station_id"] == 1)].head(11).to_dict(
    orient="list"
)
# ts_activity[(ts_activity["date"] >= "2017-07-09 00:54:05") & (ts_activity["station_id"] == 22)].head(11).to_dict(orient="list")

{'station_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'date': [Timestamp('2018-12-01 00:10:00'),
  Timestamp('2018-12-01 00:20:00'),
  Timestamp('2018-12-01 00:30:00'),
  Timestamp('2018-12-01 00:40:00'),
  Timestamp('2018-12-01 00:50:00'),
  Timestamp('2018-12-01 01:00:00'),
  Timestamp('2018-12-01 01:10:00'),
  Timestamp('2018-12-01 01:20:00'),
  Timestamp('2018-12-01 01:30:00'),
  Timestamp('2018-12-01 01:40:00'),
  Timestamp('2018-12-01 01:50:00')],
 'available_stands': [28.0,
  28.0,
  28.0,
  27.0,
  27.0,
  28.0,
  28.0,
  28.0,
  28.0,
  29.0,
  29.0],
 'available_bikes': [4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0],
 'status': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'transactions_in': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'transactions_out': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'transactions_all': [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 'consecutive_no_transactions_out': [0, 1, 2, 3, 4, 0, 1, 

In [ ]:
{
    "station_id": [22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22],
    "date": [
        pd.Timestamp("2018-12-01 00:10:00"),
        pd.Timestamp("2018-12-01 00:20:00"),
        pd.Timestamp("2018-12-01 00:30:00"),
        pd.Timestamp("2018-12-01 00:40:00"),
        pd.Timestamp("2018-12-01 00:50:00"),
        pd.Timestamp("2018-12-01 01:00:00"),
        pd.Timestamp("2018-12-01 01:10:00"),
        pd.Timestamp("2018-12-01 01:20:00"),
        pd.Timestamp("2018-12-01 01:30:00"),
        pd.Timestamp("2018-12-01 01:40:00"),
        pd.Timestamp("2018-12-01 01:50:00"),
    ],
    "available_stands": [2.0, 6.0, 9.0, 9.0, 11.0, 13.0, 13.0, 13.0, 17.0, 18.0, 20.0],
    "available_bikes": [31.0, 27.0, 24.0, 24.0, 22.0, 20.0, 20.0, 20.0, 16.0, 15.0, 13.0],
    "status": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    "transactions_in": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    "transactions_out": [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0, 4.0, 1.0, 2.0],
    "transactions_all": [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 2.0, 0.0, 4.0, 1.0, 2.0],
    "consecutive_no_transactions_out": [0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0],
}

In [33]:
data = {
    "station_id": [1] * 11 + [22] * 11,
    "date": [
        pd.Timestamp("2018-12-01 00:10:00"),
        pd.Timestamp("2018-12-01 00:20:00"),
        pd.Timestamp("2018-12-01 00:30:00"),
        pd.Timestamp("2018-12-01 00:40:00"),
        pd.Timestamp("2018-12-01 00:50:00"),
        pd.Timestamp("2018-12-01 01:00:00"),
        pd.Timestamp("2018-12-01 01:10:00"),
        pd.Timestamp("2018-12-01 01:20:00"),
        pd.Timestamp("2018-12-01 01:30:00"),
        pd.Timestamp("2018-12-01 01:40:00"),
        pd.Timestamp("2018-12-01 01:50:00"),
    ]
    + [
        pd.Timestamp("2018-12-01 00:10:00"),
        pd.Timestamp("2018-12-01 00:20:00"),
        pd.Timestamp("2018-12-01 00:30:00"),
        pd.Timestamp("2018-12-01 00:40:00"),
        pd.Timestamp("2018-12-01 00:50:00"),
        pd.Timestamp("2018-12-01 01:00:00"),
        pd.Timestamp("2018-12-01 01:10:00"),
        pd.Timestamp("2018-12-01 01:20:00"),
        pd.Timestamp("2018-12-01 01:30:00"),
        pd.Timestamp("2018-12-01 01:40:00"),
        pd.Timestamp("2018-12-01 01:50:00"),
    ],
    "available_stands": [28.0, 28.0, 28.0, 27.0, 27.0, 28.0, 28.0, 28.0, 28.0, 29.0, 29.0]
    + [2.0, 6.0, 9.0, 9.0, 11.0, 13.0, 13.0, 13.0, 17.0, 18.0, 20.0],
    "available_bikes": [4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0]
    + [31.0, 27.0, 24.0, 24.0, 22.0, 20.0, 20.0, 20.0, 16.0, 15.0, 13.0],
    "status": [1] * 11 + [1] * 11,
    "transactions_out": [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
    + [0.0, 4.0, 3.0, 0.0, 2.0, 2.0, 0.0, 0.0, 4.0, 1.0, 2.0],
    "consecutive_no_transactions_out": [0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1] + [0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0],
}

df_activite = pd.DataFrame(data)
# drop columns we want to test.
df_activite = df_activite.drop(columns=["transactions_out", "consecutive_no_transactions_out"], axis=1)

result = get_transactions_out(df_activite)
result = get_consecutive_no_transactions_out(result)

expected = pd.DataFrame(data)

pd.testing.assert_frame_equal(result, expected)

In [34]:
result

,station_id,date,available_stands,available_bikes,status,transactions_out,consecutive_no_transactions_out
0,1,2018-12-01 00:10:00,28.0,4.0,1,0.0,0
1,1,2018-12-01 00:20:00,28.0,4.0,1,0.0,1
2,1,2018-12-01 00:30:00,28.0,4.0,1,0.0,2
3,1,2018-12-01 00:40:00,27.0,5.0,1,0.0,3
4,1,2018-12-01 00:50:00,27.0,5.0,1,0.0,4
5,1,2018-12-01 01:00:00,28.0,4.0,1,1.0,0
6,1,2018-12-01 01:10:00,28.0,4.0,1,0.0,1
7,1,2018-12-01 01:20:00,28.0,4.0,1,0.0,2
8,1,2018-12-01 01:30:00,28.0,4.0,1,0.0,3
9,1,2018-12-01 01:40:00,29.0,3.0,1,1.0,0
